In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [3]:
y_preds = {}

In [4]:
# prepare training data


In [5]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=20, sample_weights_size=20, is_oversample=True, include_new_data=False)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, discretize_size=50, dis_strategy="quantile", normalize=True, scale=False,)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=50000)
feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Oversampling train users
After oversample: positive messages: 2792, negative messages: 3491
Data size: 6283

Finished windowfying
Featurizing calculate_feats=True, normalize=True, discretize=True, discretize_size=50, include_feats=['first_prons', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 6283, 6283
Data size: 4650, 4650
Calculating first prons
Calculating NSSI words
Calculating first prons
Calculating NSSI words
Normalizing features
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 4 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.loc

Is the combined the same from tfidf: False


In [6]:
# traditional classifiers

In [7]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.81      0.99      0.89      3602
           1       0.86      0.21      0.34      1048

    accuracy                           0.81      4650
   macro avg       0.84      0.60      0.62      4650
weighted avg       0.82      0.81      0.77      4650

[[3567   35]
 [ 826  222]]
{'precision': 0.7205882352941176, 'recall': 0.47115384615384615, 'F1': 0.5697674418604651, 'ERDE_5': 0.25674635415573577, 'ERDE_50': 0.1410671271845272, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.5453394026355205}


In [8]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      3602
           1       0.60      0.48      0.53      1048

    accuracy                           0.81      4650
   macro avg       0.73      0.69      0.71      4650
weighted avg       0.80      0.81      0.80      4650

[[3259  343]
 [ 541  507]]
{'precision': 0.45180722891566266, 'recall': 0.7211538461538461, 'F1': 0.5555555555555556, 'ERDE_5': 0.29840960491536916, 'ERDE_50': 0.12145040770361679, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5338998672097269}


In [9]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 3

In [10]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Evaluating
146/146 [==============================] - 2s 9ms/step - loss: 0.5024 - tp: 133.0000 - fp: 20.0000 - tn: 3582.0000 - fn: 915.0000 - accuracy: 0.7989 - precision: 0.8693 - recall: 0.1269 - f1_metric: 0.1094
Test Score: 0.5024381279945374
Test Accuracy: 133.0
146/146 [==============================] - 1s 8ms/step
Entered here
              precision    recall  f1-score   support

           0       0.80      0.99      0.88      3602
           1       0.87      0.13      0.22      1048

    accuracy                           0.80      4650
   macro avg       0.83      0.56      0.55      4650
weighted avg       0.81      0.80      0.74      4650

[[3582   20]
 [ 915  133]]
Finished training and evaluation
{'precision': 0.8205128205128205, 'recall': 0.30769230

In [11]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [12]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=100, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=lstm_model_32 and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback
Evaluating
146/146 [==============================] - 24s 152ms/step - loss: 1.1015 - tp: 340.0000 - fp: 100.0000 - tn: 3502.0000 - fn: 708.0000 - accuracy: 0.8262 - precision: 0.7727 - recall: 0.3244 - f1_metric: 0.2466
Test Score: 1.101483702659607
Test Accuracy: 340.0
146/146 [==============================] - 23s 151ms/step
Entered here
              precision    recall  f1-score   support

           0       0.83      0.97      0.90      3602
           1       0.77      0.32      0.46      1048

    accuracy                           0.83      4650
   macro avg       0.80      0.65      0.68      4650
weighted avg       0.82      0.83      0.80      4650

[[3502  100]
 [ 708  340]]
Finished training and evaluation
{'precision': 0.6352941176470588, 'recall': 0.5192307692307693, 'F1': 0.5714285714285715, 'ERDE_5': 0.26366901092866, 'ERDE_50':

In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [13]:
y_pred = ensemble_vote(np.array([y_preds["bayes"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

{'precision': 0.72, 'recall': 0.5192307692307693, 'F1': 0.6033519553072626, 'ERDE_5': 0.2578546144872604, 'ERDE_50': 0.13040926848079604, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5798331518746979}


{'precision': 0.72,
 'recall': 0.5192307692307693,
 'F1': 0.6033519553072626,
 'ERDE_5': 0.2578546144872604,
 'ERDE_50': 0.13040926848079604,
 'median_latency_tps': 11.0,
 'median_penalty_tps': 0.03898023902249159,
 'speed': 0.9610197609775084,
 'latency_weighted_f1': 0.5798331518746979}